# Notebook activity 7.5 Fitting a binomial GLM in R

In this notebook activity, we will use R to fit a binomial GLM with a logit link. We'll fit the model to data from the OU students dataset, taking `examScore` as the response and `bestPrevModScore` and `age` as explanatory variables, as we did in Activity 21 (Subsection 4.2, Unit 7).

As a reminder, these variables are described as:

* `examScore`: the final exam score, taking values from $0$ to $100$ (rounded to the nearest integer)
    
* `bestPrevModScore`:  the best previous overall final module score, taking values from $0$ to $100$ (rounded to one decimal place) 
    
* `age`: the age of the student (in years), with one of the values $-2$, $-1$, $0$, $1$ or $2$ randomly added. 

The data are stored in the data frame `ouStudents`.

## (a) Getting started

### (i)

Start by running the following code.

In [ ]:
# Load the M348 package
library(M348)
# Load the ouStudents data frame
data(ouStudents)
# Check the data frame has loaded OK
head(ouStudents)

## (b) The `glm()` function for a binomial response

As we've already seen in Unit 7, we use the `glm()` function to fit a GLM in R for a response variable $Y$ with factors $A$, $B$, ..., $Z$ and covariates $x_1$, $x_2$, ..., $x_q$ as explanatory variables, using the command:

`glm(Y ~ A + B + ... + Z + x1 + x2 + ... + xq, family = "name of a distribution", data = dataframe)`. 

Unsurprisingly, when we have a binomial distribution for the response, `family` is set to `binomial`.

We've already used `binomial` for `family` when using logistic regression in Unit 6. But, if the Bernoulli and binomial are both specified by using `family = binomial`, how does R know whether the response is Bernoulli or binomial?

Well, R can tell whether `binomial` refers to a Bernoulli GLM or a binomial GLM, because the data for the two response variables are in different forms. 

* A response with a Bernoulli distribution, is simply a single column of 0s and 1s, which R can recognise as Bernoulli data within the binomial family. 

* For a response with a binomial distribution, it is necessary to have **two** columns of data for the response - the first column is for the number of 'successes' for each binomial observation, and the second column is for the number of 'failures'. 

So, the first thing that we need to do before we can fit a binomial GLM for `examScore`, is to get our response data into the correct format!

## (c) Specifying data for a binomial GLM

The initial step in fitting a binomial GLM for a response variable $Y$, involves preparing the response data so that the `glm()` function for the binomial family works properly in R. 

Because the binomial distribution for $Y$ has two parameters ($N_i$ and $p_i$), R requires the binomial data to be in a **matrix format** with two columns, one containing the number of 'successes', and the other containing the number of 'failures'. 

Suppose that the successes are stored in the vector `Y`, and the number of trials for each observation are stored in the vector `N`. Then we want our response data matrix to have `Y` as its first column, and `N - Y` as its second column. 

We can create a matrix with this structure by using the `cbind()` function which *binds* columns of vectors together. The required command to do this is:

`cbind(Y, N - Y)`.

In the context of fitting a binomial GLM for `examScore` using the data in the `ouStudents` data frame, the 'successes' are stored in `examScore`. 

Data on the number of 'failures' are not given directly in the data frame, but we *do* know that the number of trials for each observation is 100, since there are 100 marks for each exam. Therefore, the 'failures' can be calculated as `100 - examScore`.

### (i)

Run the code cell below to create and add to `ouStudents` a matrix called `examScoreMatrix` with the data in `examScore` in its first column and `100 - examScore` in its second column and then display the first few rows of `ouStudents`.

In [ ]:
ouStudents$examScoreMatrix <- cbind(ouStudents$examScore, 100 - ouStudents$examScore)
head(ouStudents)

#### Solution <a class="tocSkip">

Note that in the table `examScoreMatrix` is shown as a single column where every entry has two numbers separated by a comma (and these two numbers add up to 100).

## (d) Fitting a binomial GLM

Once the response data are in the required matrix form, we can proceed to fit a binomial GLM.

Returning to the general case, the response variable in the `glm()` function is now a **matrix** with `Y` in the first column, and `N - Y` in the second column. The `glm()` function is otherwise used as for logistic regression (for Bernoulli data). So that if the data matrix is called `yMatrix`, say, then we use the command:

`glm(yMatrix ~ x, family = binomial, data = dataframe)`.

Recall that the `glm()` function will use the canonical link function for the family specified. So, for this model, R will use the logit link.

In the previous code cell, you created the data matrix required for the `glm()` function for modelling `examScore`: this matrix was called `examScoreMatrix`.

### (i)

In the empty code cell below, fit a binomial GLM for the response data stored in `examScoreMatrix` and explanatory variables `bestPrevmodScore` and `age` storing the results in an object called `examScoreFit`. Obtain a summary of the results from fitting the model.

Confirm that the resulting parameter estimates (rounded to four decimal places) are the same as those given in Table 7 of Activity 21 (Subsection 4.2).

#### Solution <a class="tocSkip">

The required code and resulting output are given below.

The parameter estimates (rounded to four decimal places) are indeed the same as those given in Table 7 of Activity 21 (Subsection 4.2, Unit 7).

In [3]:
examScoreFit <- glm(examScoreMatrix ~ bestPrevModScore + age, 
                    family = binomial, data = ouStudents)
summary(examScoreFit)


Call:
glm(formula = examScoreMatrix ~ bestPrevModScore + age, family = binomial, 
    data = ouStudents)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-15.5478   -2.1775    0.6821    2.8365   11.5252  

Coefficients:
                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)      -3.3386601  0.0537657 -62.096   <2e-16 ***
bestPrevModScore  0.0471885  0.0006050  78.001   <2e-16 ***
age              -0.0039868  0.0004384  -9.094   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 31830  on 1795  degrees of freedom
Residual deviance: 25381  on 1793  degrees of freedom
AIC: 33687

Number of Fisher Scoring iterations: 4


## (e) Predicted mean responses

In a binomial GLM, the predicted mean response for a new response $Y_0$ is calculated as

\begin{equation*}\widehat{\mu}_0 = N_0 \times \widehat{p}_0,\end{equation*}

where $N_0$ is the number of trials for the new response and $\widehat{p}_0$ is the predicted 'success' probability for the new response.

When we have a binomial GLM, the `predict()` function calculates the predicted 'success' probability $\widehat{p}_0$, rather than the predicted mean response $\widehat\mu_0$. So, we then need to multiply our predicted 'success' probability by $N_0$ to get the predicted mean response.

You will see this yourself by using the `predict()` function for the new response given in part (d) of Activity 21 (Subsection 4.2, Unit 7), which has a value of 79.2 for `bestPrevModScore` and 64 for `age`. 

### (i)

Use the empty code cell below to create a data frame called `xNew` storing the values of 79.2 for `bestPrevModScore` and 64 for `age` for this new student. Check that `xNew` has been constructed correctly by getting R to display the data frame.

(You might find it useful to look back at Notebook activity 7.2.)

#### Solution <a class="tocSkip">

The required code and resulting output are given below.

The data frame has indeed been constructed correctly.

In [4]:
xNew <- data.frame(bestPrevModScore = 79.2, age = 64)
xNew

bestPrevModScore age
1 79.2             64

### (ii)

Run the code below to use the `predict()` function for this new student. 

In [ ]:
predict(examScoreFit, newdata = xNew, type = "response")

#### Solution <a class="tocSkip">

Running the code gives a value of 0.5358 (to four decimal places) for the prediction.

### (iii)

By comparing the prediction you obtained in part (ii) with the solution to part (d) of Activity 21 (Subsection 4.2, Unit 7), confirm that `predict()` has calculated the predicted 'success' probability $\widehat{p}_0$ and *not* the predicted mean response $\widehat{\mu}_0$.

#### Solution <a class="tocSkip">

In the solution to part (d) of Activity 21 (Subsection 4.2), the value of the predicted 'success' probability $\widehat{p}_0$ was calculated to be 0.5358, while that for the predicted mean response $\widehat{\mu}_0$ was 53.58.

The value calculated using `predict()` in part (ii) therefore is indeed $\widehat{p}_0$ rather that $\widehat{\mu}_0$.